# Latent Dirichlet Allocation using TensorFlow (Batch VB)

In [1]:
import tensorflow as tf
import numpy as np
import tensorflow.contrib.eager as tfe

print(tf.__version__)
print(tf.__git_version__)

tfe.enable_eager_execution()

1.6.0
v1.6.0-0-gd2e24b6039


In [2]:
res = tf.meshgrid(list(range(10)), [1], indexing="ij")
res = tf.squeeze(tf.stack(res, axis=1), axis=2)
res

<tf.Tensor: id=18, shape=(10, 2), dtype=int32, numpy=
array([[0, 1],
       [1, 1],
       [2, 1],
       [3, 1],
       [4, 1],
       [5, 1],
       [6, 1],
       [7, 1],
       [8, 1],
       [9, 1]], dtype=int32)>

In [3]:
ref = tf.fill(value=1.0, dims=(10, 3))
ref = tfe.Variable(ref)
upd = tf.fill(value=-1.0, dims=(10, ))
print(ref)
print(tf.scatter_nd_update(ref=ref, indices=res, updates=upd))
print(ref)

<tf.Variable 'Variable:0' shape=(10, 3) dtype=float32, numpy=
array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]], dtype=float32)>
tf.Tensor(
[[ 1. -1.  1.]
 [ 1. -1.  1.]
 [ 1. -1.  1.]
 [ 1. -1.  1.]
 [ 1. -1.  1.]
 [ 1. -1.  1.]
 [ 1. -1.  1.]
 [ 1. -1.  1.]
 [ 1. -1.  1.]
 [ 1. -1.  1.]], shape=(10, 3), dtype=float32)
<tf.Variable 'Variable:0' shape=(10, 3) dtype=float32, numpy=
array([[ 1., -1.,  1.],
       [ 1., -1.,  1.],
       [ 1., -1.,  1.],
       [ 1., -1.,  1.],
       [ 1., -1.,  1.],
       [ 1., -1.,  1.],
       [ 1., -1.,  1.],
       [ 1., -1.,  1.],
       [ 1., -1.,  1.],
       [ 1., -1.,  1.]], dtype=float32)>



### Latent Dirichlet Allocation model:

 - $\theta_{d=1,...,M} \sim \mathrm{Dir}_K(\alpha)$
 - $\beta_{k=1,...,K} \sim \mathrm{Dir}_V(\eta)$
 - $z_{d=1,...,M,i=1,...,N_d} \sim \mathrm{Multinomial}_{ \ K}(\theta_d)$
 - $w_{d=1,...,M,i=1,...,N_d} \sim \mathrm{Multinomial}_{ \ V}(\beta_{z_{di}})$



where

 - $V$ is a vocabulary size
 - $K$ is a number of topics
 - $\eta$ is the parameter of the Dirichlet prior on the per-topic word distribution (known and symetric)
 - $\alpha$ is the parameter of the Dirichlet prior on the per-document topic distributions (known and symetric)
 - $\theta_d$ is the topic distribution for document d
 - $\beta_k$ is the word distribution for topic k
 - $z_{di}$ is the topic for the i-th word in document d
 - $w_{di}$ is a specific word from d-th document belonging to $V$

In plate notation:

![hustlin_erd](LDA_plate_notation.png)

### Posterior distributions:

We assume full factorial design:


$$q(z, \beta, \theta) = \prod_d \prod_i q(z_{di}) \times \prod_d q(\theta_d) \times \prod_k q(\beta_k)$$

where

- $q(z_{di}) = p(z_{di}|\phi) = \mathrm{Multinomial}_{ \ K}(\phi_{dw_{di}}) $
- $q(\theta_d) = p(\theta_d|\gamma) = \mathrm{Dir}_{K}(\gamma_{d}) $
- $q(\beta_k) = p(\beta_k|\lambda) = \mathrm{Dir}_{V}(\lambda_{k}) $

### Parameters Tensors:

*Vocabulary to topic membership:*

$$ \phi = \phi_{d=1,...,D, \ v=1,...,V, \ k=1,...,K} \in \mathbb{R}_{+}^{D \times V \times K}$$
$ \forall_{d \in 1,...,D} \ \phi_d $ is a *stochastic matrix*.

*Distribution of topics:*
$$ \gamma = \gamma_{d=1,...,D, \ k=1,...,K} \in \mathbb{R}_{+}^{D \times K}$$

*Within topic vocabulary profile:*

$$ \lambda = \lambda_{k=1,...,K \ v=1,...,V} \in \mathbb{R}_{+}^{K \times V} $$

### ELBO:

From the definition of the **E**vidence **L**ower **Bo**und:

$$ \mathcal{L}(w, \phi, \gamma, \lambda) := \mathbb{E}_{z, \theta, \beta}\left[\log p(w, z, \theta, \beta| \alpha, \eta) \right] - \mathbb{E}_{z, \theta, \beta}\left[\log q(z, \theta, \beta) \right]$$

Using the probability factorization of the LDA model

$$ \log p(w, z, \theta, \beta| \alpha, \eta) = \log p(w|z, \beta) p(z|\theta) p(\theta | \alpha) p(\beta | \eta) = $$

$$ = \log \left( \prod_{k=1}^K p(\beta_k|\eta) \times \prod_{d=1}^D p(\theta_d|\alpha) \times \prod_{d=1}^{D} \prod_{i=1}^{N_d} p(w_{di}|\beta_{z_{di}}) p(z_{di}|\theta_d) \right) = $$

$$ =  \sum_{k=1}^K \log p(\beta_k|\eta) +  \sum_{d=1}^D \log p(\theta_d|\alpha) + \sum_{d=1}^{D} \sum_{i=1}^{N_d} \log p(w_{di}|\beta_{z_{di}}) p(z_{di}|\theta_d) = $$

$$ =  \sum_{k=1}^K \log p(\beta_k|\eta) +  \sum_{d=1}^D \left( \log p(\theta_d|\alpha) + \sum_{i=1}^{N_d} \log p(w_{di}|\beta_{z_{di}}) p(z_{di}|\theta_d) \right) = $$

$$ =  \sum_{d=1}^D \left( \log p(\theta_d|\alpha) + \sum_{i=1}^{N_d} \log \left( p(w_{di}|\beta_{z_{di}}) p(z_{di}|\theta_d) \right) + \frac{1}{D} \sum_{k=1}^K \log p(\beta_k|\eta) \right) $$


Taking the expectations with respect to the posterior distribution results in:

$$ \mathbb{E}_{\theta} \left[ \log p(\theta_d|\alpha) \right] = C_1(\alpha) + \sum_{k=1}^K (\alpha_k - 1) \mathbb{E}_{\theta} \left[ \log \theta_{dk} \right] $$

$$ \mathbb{E}_{z, \beta} \left[ \log p(w_{di}|\beta_{z_{di}}) \right] =  \mathbb{E}_{z, \beta} \left[ \log \beta_{z_{di} i} \right] = \sum_{k=1}^{K} \phi_{dw_{di}k} \mathbb{E}_{\beta} \left[ \log \beta_{k i} \right] $$

$$ \mathbb{E}_{z, \theta} \left[ \log p(z_{di}|\theta_d) \right] =  \sum_{k=1}^{K} \phi_{dw_{di}k} \mathbb{E}_{\theta} \left[ \log \theta_{d k} \right] $$

$$ \mathbb{E}_{\beta} \left[ \log p(\beta_{k}|\eta) \right] = C_2(\eta) + \sum_{v=1}^{V} (\eta_v - 1) \mathbb{E}_{\beta} \left[ \log \beta_{kv} \right] $$

For the entropy of the posterior distribution we have:

$$ \log\left(\prod_{d=1}^{D} \prod_{i=1}^{N_{d}} q(z_{di}) \times \prod_{d=1}^D q(\theta_d) \times \prod_{k=1}^{K} q(\beta_k)\right) =  $$

$$ = \sum_{d=1}^{D} \left( \sum_{i=1}^{N_{d}} \log q(z_{di}) + \log q(\theta_d) + \frac{1}{D} \sum_{k=1}^{K} \log q(\beta_k)\right) $$

and expectations with respect to the posterior distribution:

$$ \mathbb{E}_{z} \left[ \log q(z_{di})  \right] = \mathbb{E}_{z_{di}} \left[ \mathbb{1}_{[z_{di}=k]} \log \phi_{dw_{di}k}  \right] = \sum_{k=1}^{K} \phi_{dw_{di}k} \log \phi_{dw_{di}k} $$

$$ \mathbb{E}_{\theta} \left[ \log q(\theta_d)  \right] = \mathbb{E}_{\theta} \left[ \log \frac{1}{B(\gamma_d)} \prod_{k=1}^{K} \theta_{dk}^{\gamma_{dk} - 1} \right] = -\log B(\gamma_d) + \sum_{k=1}^K (\gamma_{dk} - 1)\mathbb{E}_{\theta} \left[ \log \theta_{dk} \right]$$

$$ \mathbb{E}_{\beta}\left[ \log q(\beta_k) \right] = -\log B(\lambda_k) + \sum_{v=1}^V (\lambda_{kv} - 1)\mathbb{E}_{\beta} \left[ \log \beta_{kv} \right]$$

*We should then be able to collapse ELBO to the formula based on counts of words rather than words from documents. This will show that the LDA model is fixed size method.* 

*"When training an LDA model, you start with a collection of documents and each of these is represented by a fixed-length vector (bag-of-words). LDA is a general Machine Learning (ML) technique, which means that it can also be used for other unsupervised ML problems where the input is a collection of fixed-length vectors and the goal is to explore the structure of this data."* (Data Camp)

Grouping terms by corresponding parameters we get:

#### Document vocabulary membership

$$ \mathcal{L}_{[\phi_{dv}]} = \sum_{d=1}^{D} \sum_{i=1}^{N_d} \sum_{k=1}^K \phi_{dw_{di}k} \left( \mathbb{E}_{\beta} \left[ \log \beta_{k i} \right] + \mathbb{E}_{\theta} \left[ \log \theta_{d k} \right] - \log \phi_{dw_{di}k}  \right) = $$

$$ = \sum_{d=1}^{D} \sum_{v=1}^{V} \sum_{k=1}^K n_{dv} \phi_{dvk} \left( \mathbb{E}_{\beta} \left[ \log \beta_{k v} \right] + \mathbb{E}_{\theta} \left[ \log \theta_{d k} \right] - \log \phi_{dvk}  \right)$$

where $n_{dv}$ is a number of times word $v$ from vocabulary $V$ was counted in document $d$.

#### Distribution of topics

$$ \mathcal{L}_{[\gamma_{d}]} = \sum_{d=1}^{D} \sum_{k=1}^K \left\{ (\alpha_k - 1) \mathbb{E}_{\theta} \left[ \log \theta_{dk} \right] + \log B(\gamma_d) - (\gamma_{dk} - 1)\mathbb{E}_{\theta} \left[ \log \theta_{dk} \right] \right\} = $$

$$ = \sum_{d=1}^{D} \sum_{k=1}^K \left\{ (\alpha_k - \gamma_{dk}) \mathbb{E}_{\theta} \left[ \log \theta_{dk} \right] + \frac{1}{K} \log B(\gamma_d) \right\}$$

#### Topic vocabulary profile

$$ \frac{1}{D} \sum_{d=1}^{D} \sum_{v=1}^{V} \sum_{k=1}^K \left\{ (\eta_v - \lambda_{kv}) \mathbb{E}_{\beta} \left[ \log \beta_{k} \right] + \frac{1}{V} \log B(\lambda_k) \right\}$$

### Coordinate ascent optimization

Optimizing for every document and vocabulary entry separately we get:

#### Document vocabulary membership:

$$ \mathrm{argmax}_{\phi_{dv} \in \Delta^{K-1}} \mathcal{L}_{[\phi_{dv}]} =  \mathrm{argmax}_{\phi_{dv} \in \Delta^{K-1}} \sum_{k=1}^K n_{dv} \phi_{dvk} \left( \mathbb{E}_{\beta} \left[ \log \beta_{k v} \right] + \mathbb{E}_{\theta} \left[ \log \theta_{d k} \right] - \log \phi_{dvk}  \right) = $$

$$ =\mathrm{argmax}_{\phi_{dv} \in \mathbb{R}_{+}^K} \sum_{k=1}^K n_{dv} \phi_{dvk} \left( \mathbb{E}_{\beta} \left[ \log \beta_{k v} \right] + \mathbb{E}_{\theta} \left[ \log \theta_{d k} \right] - \log \phi_{dvk}  \right) + \xi (\sum_{k=1}^K \phi_{dvk} - 1) = \Psi(\phi_{dv}, \xi) $$

$$ \frac{\partial}{\partial \phi_{dvk}} \Psi(\phi_{dv}, \xi) = n_{dv} \left( \mathbb{E}_{\beta} \left[ \log \beta_{k v} \right] + \mathbb{E}_{\theta} \left[ \log \theta_{d k} \right] - \log \phi_{dvk} - 1 \right) + \xi $$

$$ \frac{\partial}{\partial \xi} \Psi(\phi_{dv}, \xi) = \sum_{k=1}^K \phi_{dvk} - 1 $$

$$ \frac{\partial}{\partial \phi_{dvk}} \Psi(\phi_{dv}, \xi) = 0 \iff n_{dv} \left( \mathbb{E}_{\beta} \left[ \log \beta_{k v} \right] + \mathbb{E}_{\theta} \left[ \log \theta_{d k} \right] - \log \phi_{dvk} - 1 \right) + \xi = 0 \iff $$

$$ \iff \mathbb{E}_{\beta} \left[ \log \beta_{k v} \right] + \mathbb{E}_{\theta} \left[ \log \theta_{d k} \right] - \log \phi_{dvk} - 1 + \xi' = 0 \iff $$

$$ \iff \log \phi_{dvk} = \mathbb{E}_{\beta} \left[ \log \beta_{k v} \right] + \mathbb{E}_{\theta} \left[ \log \theta_{d k} \right] - 1 + \xi'  \iff $$

$$ \iff \phi_{dvk} \propto \exp \left\{ \mathbb{E}_{\beta} \left[ \log \beta_{k v} \right] + \mathbb{E}_{\theta} \left[ \log \theta_{d k} \right] \right\} $$

#### Doucuments' distribution of topics

Since

$$ \mathbb{E}_{\theta} \left[ \log \theta_{dk} \right] = \psi(\gamma_{dk}) - \psi(\sum_{k=1}^K\gamma_{dk})$$

and

$$ \log B(\gamma_d) = \log \frac{\prod_{k=1}^{K}\Gamma(\gamma_{dk})}{\Gamma ({\sum_{k=1}^K \gamma_{dk}})} = \sum_{k=1}^{K}\log \Gamma (\gamma_{dk}) - \log \Gamma ({\sum_{k=1}^K \gamma_{dk}})$$

$$ \mathrm{argmax}_{\gamma_{d} \in \mathbb{R}_{+}^{K}} \mathcal{L}_{[\gamma_{d}]} = \mathrm{argmax}_{\gamma_{d} \in \mathbb{R}_{+}^{K}} \sum_{k=1}^K \left\{ (\alpha_k - \gamma_{dk} + \sum_{v=1}^V n_{dv} \phi_{dvk}) \mathbb{E}_{\theta} \left[ \log \theta_{dk} \right] \right\} + \log B(\gamma_d) = $$

$$ = \mathrm{argmax}_{\gamma_{d} \in \mathbb{R}_{+}^{K}} \sum_{k=1}^K \left\{ (\alpha_k - \gamma_{dk} + \sum_{v=1}^V n_{dv} \phi_{dvk}) (\psi(\gamma_{dk}) - \psi(\sum_{k=1}^K\gamma_{dk})) \right\} + \sum_{k=1}^{K}\log \Gamma (\gamma_{dk}) - \log \Gamma ({\sum_{k=1}^K \gamma_{dk}}) = \Psi(\gamma_d)$$

$$ \frac{\partial}{\partial \gamma_{dj}} \Psi(\gamma_d) = -\psi(\gamma_{dj}) + \psi({\sum_k \gamma_{dk})} + \sum_{k=1}^K \left\{ (\alpha_k - \gamma_{dk} + \sum_{v=1}^V n_{dv} \phi_{dvk})*(\psi'(\gamma_{dj})*1_{k=j} - \psi'(\sum_{k=1}^K\gamma_{dk})) \right\} + \psi(\gamma_{dj}) - \psi(\sum_k \gamma_{dk}) = $$

$$ = \sum_{k=1}^K \left\{ (\alpha_k - \gamma_{dk} + \sum_{v=1}^V n_{dv} \phi_{dvk})*(\psi'(\gamma_{dj})*1_{k=j} - \psi'(\sum_{k=1}^K\gamma_{dk})) \right\} $$

Since *trigamma* function has no real roots it follows that:

$$ \frac{\partial}{\partial \gamma_{dj}} \Psi(\gamma_d) = 0 \iff \forall_{k} \ \ \  \alpha_k - \gamma_{dk} + \sum_{v=1}^V n_{dv} \phi_{dvk} = 0 \iff $$

$$ \iff   \gamma_{dk} = \alpha_k  + \sum_{v=1}^V n_{dv} \phi_{dvk} $$

#### Topic vocabulary profile (M-Step)

Exact derivation as for the $\gamma_d$ gives an update equation for $\lambda_k$

$$ \lambda_{kv} = \eta_v + \sum_{d=1}^D n_{dv}\phi_{dvk} $$

### Final algorithm

 - initialize $\lambda$ and $\gamma$
 - while improvement $ \mathcal{L}(w, \phi, \gamma, \lambda) > 1e-6$ 
     - for $d = 1,...,D$
         - $\phi_{dvk} \propto \exp \left\{ \mathbb{E}_{\beta} \left[ \log \beta_{k v} \right] + \mathbb{E}_{\theta} \left[ \log \theta_{d k} \right] \right\}$
         - $\gamma_{dk} = \alpha  + \sum_{v=1}^V n_{dv} \phi_{dvk}$
     - $\lambda_{kv} = \eta_v + \sum_{d=1}^D n_{dv}\phi_{dvk}$

### Generate data

In [4]:
# set data dimensions
K = 3
V = 10
D = 1000
N = 100

# set the seed
np.random.seed(2014)

# beta prior parameters
eta = np.ones(V) * 1e-1

# beta profiles
beta = np.random.dirichlet(alpha=eta, size=K)

# theta prior parameters
alpha = np.ones(K) * 1e-1
# alpha[0] = 10

# document's prior topic allocation
theta = np.random.dirichlet(alpha=alpha, size=D)

# word's topic membership
z = [np.random.choice(K, size=N, replace=True, p=theta[d, :]) for d in range(D)]
z = np.vstack(z)

# actual words and counts
w = [np.array([np.random.choice(V, size=1, p=beta[k,:])[0] for k in z[d, :]]  + list(range(V))) for d in range(D)]
nw = [np.unique(w[d], return_counts=True)[1] for d in range(D)]
nw = np.vstack(nw)
w = np.vstack(w)

nw = tf.convert_to_tensor(nw, dtype=tf.float32)
# nw

In [5]:
z.shape

(1000, 100)

### Tensorflow model

Few notes:
- there is a tape now for gradients like in pyTorch
- we will only use tensorflow for fast tensor operations

In [6]:
# initialize LDA parameters
def initialize_parameters(K, V, D, alpha=1e-1, eta=1e-1, seed=2014):
    """
    Initialize parameters of LDA model returning adequate Tensors.

    args:
    
        K (int): number of LDA components 
        V (int): vocabulary size
        D (int): number of documents
        alpha (float): hyperparameter for theta prior
        eta (float): hyperparameter for beta prior
       
       
    returns:
    
        eta: [1 x V] tensor with prior parameters (alpha) for beta
        lambda: k x [1, V] tensors with posterior word distribution per class
        phi: [D, V, K] tensor with vocabulary membership per document
        gamma: k x [D, 1] tensors
        
    """
    tf.set_random_seed(seed)
    
    eta = tf.ones((1, V)) * eta
    alpha = tf.ones((K, 1)) * alpha
    
    lambda_ = [tf.abs(tf.random_normal(shape=(1, V))) for k in range(K)]
    
    phi = tf.random_normal(shape=(D, V, K))
    phi = tf.nn.softmax(phi, axis=2)
    
    gamma = [tf.abs(tf.random_normal(shape=(D, 1))) for k in range(K)]
    
    return eta, alpha, lambda_, phi, gamma

# test
eta, alpha, lambda_, phi, gamma = initialize_parameters(K, V, D)
eta


<tf.Tensor: id=47, shape=(1, 10), dtype=float32, numpy=array([[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]], dtype=float32)>

In [7]:
# lambda update
def update_lambda(lambda_, eta, phi, nw):
    
    K = len(lambda_)
    for k in range(K):
        lambda_[k] = tf.add(tf.reduce_sum(tf.multiply(phi[:,:,k], nw), axis=0, keepdims=True), eta)
        
    return lambda_ 

# test
# print(lambda_)
# update_lambda(lambda_, eta, phi, nw)
# print(lambda_)

In [8]:
# gamma update
def update_gamma(gamma, alpha, phi, nw):
    
    K = len(gamma)
    for k in range(K):
        gamma[k] = tf.add(tf.reduce_sum(tf.multiply(phi[:,:,k], nw), axis=1, keepdims=True), alpha[k])
        
    return gamma

# test
# print(gamma)
# update_gamma(gamma, alpha, phi, nw)
# print(gamma)

$$ \mathbb{E}_{\beta} \left[ \log \beta_{k v} \right]  = \psi(\lambda_{kv}) - \psi(\sum_{v=1}^V \lambda_{kv}) $$

In [9]:
def e_log_beta(lambda_):
    return [tf.digamma(lam) - tf.digamma(tf.reduce_sum(lam)) for lam in lambda_]

# e_log_beta(lambda_)

$$ \mathbb{E}_{\theta} \left[ \log \theta_{dk} \right] = \psi(\gamma_{dk}) - \psi(\sum_{k=1}^K\gamma_{dk})$$

In [10]:
def e_log_theta(gamma):
    return [tf.digamma(g_k) - tf.digamma(tf.reduce_sum(g_k)) for g_k in gamma]

# e_log_theta(gamma)

$$ \phi_{dvk} \propto \exp \left\{ \mathbb{E}_{\beta} \left[ \log \beta_{k v} \right] + \mathbb{E}_{\theta} \left[ \log \theta_{d k} \right] \right\} $$

In [11]:
def update_phi(lambda_, gamma, D, V):
    eb_1xvxk = tf.stack(e_log_beta(lambda_), axis=2)
    eb_dxvxk = tf.tile(eb_1xvxk, multiples=[D, 1, 1])
    et_dx1xk = tf.stack(e_log_theta(gamma), axis=2)
    et_dxvxk = tf.tile(et_dx1xk, multiples=[1, V, 1])
    phi_dxvxk = tf.nn.softmax(logits=eb_dxvxk + et_dxvxk, axis=2)
    return phi_dxvxk

# update_phi(lambda_, gamma, D, V)

### ELBO

$$ A := \sum_{d=1}^{D} \sum_{v=1}^{V} \sum_{k=1}^K n_{dv} \phi_{dvk} \left( \mathbb{E}_{\beta} \left[ \log \beta_{k v} \right] + \mathbb{E}_{\theta} \left[ \log \theta_{d k} \right] - \log \phi_{dvk}  \right)$$

$$ B := \sum_{d=1}^{D} \sum_{k=1}^K \left\{ (\alpha_k - \gamma_{dk}) \mathbb{E}_{\theta} \left[ \log \theta_{dk} \right] + \frac{1}{K} \log B(\gamma_d) \right\}$$

In [12]:
def elbo(lambda_, gamma, D, V, nw):
    eb_1xvxk = tf.stack(e_log_beta(lambda_), axis=2)
    eb_dxvxk = tf.tile(eb_1xvxk, multiples=[D, 1, 1])
    et_dx1xk = tf.stack(e_log_theta(gamma), axis=2)
    et_dxvxk = tf.tile(et_dx1xk, multiples=[1, V, 1])
    phi_dxvxk = tf.nn.softmax(logits=eb_dxvxk + et_dxvxk, axis=2)
    n_dxvxk = tf.tile(tf.expand_dims(nw / tf.reduce_sum(nw), axis=2), multiples=[1, 1, K])
    
    A = tf.reduce_sum(n_dxvxk * phi_dxvxk * (eb_dxvxk + et_dxvxk - tf.log(phi_dxvxk + 1e-6)))
    
    
    
    return A.numpy()

elbo(lambda_, gamma, D, V, nw)    

-10.225006

### Wrapping it all up together

In [13]:
seed = 1

In [14]:
seed += 1
eta, alpha, lambda_, phi, gamma = initialize_parameters(K, V, D, seed=seed)

prev_elbo = 0.0
next_elbo = 0.0
iter = 0

for i in range(100000):
    
    for j in range(100000):
        # E-Step:
        phi = update_phi(lambda_, gamma, D, V)
        gamma_next = update_gamma(gamma, alpha, phi, nw)
        
        diff = np.mean([tf.reduce_sum(tf.abs(g_prev - g_next)).numpy() for g_prev, g_next in zip(gamma, gamma_next)])
        gamma = gamma_next
        if diff < 1e-6:
            break
    
    # M-Step:
    lambda_ = update_lambda(lambda_, eta, phi, nw)
    
    
    next_elbo = elbo(lambda_, gamma, D, V, nw)
    print("Iteration:", iter, "ELBO:", next_elbo)
    
    diff = np.abs(next_elbo - prev_elbo)
    if diff < 1e-6:
        print("Converged!")
        break
    else:
        iter += 1
        prev_elbo = next_elbo


Iteration: 0 ELBO: -7.61189
Iteration: 1 ELBO: -7.5102015
Iteration: 2 ELBO: -7.4390783
Iteration: 3 ELBO: -7.385777
Iteration: 4 ELBO: -7.347884
Iteration: 5 ELBO: -7.323457
Iteration: 6 ELBO: -7.308981
Iteration: 7 ELBO: -7.301273
Iteration: 8 ELBO: -7.297392
Iteration: 9 ELBO: -7.295474
Iteration: 10 ELBO: -7.2944207
Iteration: 11 ELBO: -7.293814
Iteration: 12 ELBO: -7.293473
Iteration: 13 ELBO: -7.2932143
Iteration: 14 ELBO: -7.292981
Iteration: 15 ELBO: -7.2927957
Iteration: 16 ELBO: -7.2926826
Iteration: 17 ELBO: -7.2926016
Iteration: 18 ELBO: -7.292543
Iteration: 19 ELBO: -7.2924767
Iteration: 20 ELBO: -7.292369
Iteration: 21 ELBO: -7.292183
Iteration: 22 ELBO: -7.292083
Iteration: 23 ELBO: -7.2920647
Iteration: 24 ELBO: -7.2920504
Iteration: 25 ELBO: -7.2920423
Iteration: 26 ELBO: -7.292046
Iteration: 27 ELBO: -7.2920423
Iteration: 28 ELBO: -7.292036
Iteration: 29 ELBO: -7.292025
Iteration: 30 ELBO: -7.291994
Iteration: 31 ELBO: -7.291955
Iteration: 32 ELBO: -7.2919455
Iteratio

In [15]:
print(np.round(beta, decimals=3))
print(np.round(np.vstack([(lam / tf.reduce_sum(lam)).numpy()  for lam in lambda_]), decimals=3))

[[0.    0.    0.    0.063 0.    0.927 0.001 0.008 0.    0.   ]
 [0.579 0.01  0.    0.003 0.033 0.006 0.363 0.    0.    0.006]
 [0.041 0.    0.    0.001 0.46  0.079 0.    0.086 0.333 0.   ]]
[[0.487 0.017 0.01  0.019 0.    0.126 0.315 0.009 0.004 0.015]
 [0.011 0.009 0.009 0.06  0.064 0.757 0.009 0.026 0.045 0.009]
 [0.121 0.01  0.009 0.011 0.38  0.07  0.061 0.072 0.255 0.01 ]]


### Calculate results for visualisation of topics

https://pyldavis.readthedocs.io/en/latest/modules/API.html#pyLDAvis.prepare

In [16]:
# topic term distribution:
topic_term_dist = np.round(np.vstack([(lam / tf.reduce_sum(lam)).numpy()  for lam in lambda_]), decimals=3)
topic_term_dist

# doc_topic_dists :array-like, shape (n_docs, n_topics)
doc_topic_dist = tf.stack([tf.reshape(g_k, shape=(1000, )) for g_k in gamma], axis=1)
doc_topic_dist = doc_topic_dist / tf.reduce_sum(doc_topic_dist, axis=1, keep_dims=True)
doc_topic_dist = doc_topic_dist.numpy()

# doc_lengths :array-like, shape n_docs
doc_len = tf.reduce_sum(nw, axis=1)
doc_len = doc_len.numpy()

# vocab :array-like, shape n_terms
vocab = np.array(list(range(V)))

# term_frequency :array-like, shape n_terms
term_frec = tf.reduce_sum(nw, axis=0)
term_frec = term_frec.numpy()

In [402]:
import pyLDAvis
topics = pyLDAvis.prepare(topic_term_dist, doc_topic_dist, doc_len, vocab, term_frec)

/Users/qba/.local/share/virtualenvs/LDA-pHQnYWLy/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


In [405]:
pyLDAvis.save_html(topics, fileobj="results.html")

### Case study

#### Data Set Information:

The dataset is in the form of a 11463 x 5812 matrix of word counts, containing 11463 words and 5811 NIPS conference papers (the first column contains the list of words). Each column contains the number of times each word appears in the corresponding document. The names of the columns give information about each document and its timestamp in the following format: Xyear_paperID. 

The matrix of word counts was obtained using the R package 'tmâ€ to process the raw .txt files of the full text of the NIPS conference papers published between 1987 and 2015. The document-term matrix was constructed after tokenization, removal of stopwords and truncation of the vocabulary by only keeping words occurring more than 50 times.



In [1]:
# download dataset
! curl https://archive.ics.uci.edu/ml/machine-learning-databases/00371/NIPS_1987-2015.csv > NIPS_1987-2015.csv 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  127M  100  127M    0     0  17.2M      0  0:00:07  0:00:07 --:--:-- 22.9M


In [17]:
import pandas as pd

In [18]:
data = pd.read_csv("NIPS_1987-2015.csv")

In [19]:
nw = np.array(data.iloc[:, 1:])
nw = nw.astype('float32')
nw = nw.transpose()

In [20]:
nw = tf.convert_to_tensor(nw)
nw = nw[0:1000, :]

In [21]:
K = 10
D, V = nw.shape

In [22]:
import time
import cProfile

In [23]:
seed = 0
eta, alpha, lambda_, phi, gamma = initialize_parameters(K, V, D, seed=seed)

In [29]:
start = time.time()

def lda(eta=eta, alpha=alpha, lambda_=lambda_, phi=phi, gamma=gamma, K=K, V=V, D=D, seed=seed):

    prev_elbo = 0.0
    next_elbo = 0.0
    iter = 0

    for i in range(1):

        for j in range(1):
            # E-Step:
            phi = update_phi(lambda_, gamma, D, V)
            gamma_next = update_gamma(gamma, alpha, phi, nw)

            diff = 0.0
            diff = np.mean([tf.reduce_sum(tf.abs(g_prev - g_next)).numpy() for g_prev, g_next in zip(gamma, gamma_next)])
            gamma = gamma_next
            if diff < 1e-6:
                break

        # M-Step:
        lambda_ = update_lambda(lambda_, eta, phi, nw)


        next_elbo = elbo(lambda_, gamma, D, V, nw)
        print("Iteration:", iter, "ELBO:", next_elbo)

        diff = np.abs(next_elbo - prev_elbo)
        if diff < 1e-6:
            print("Converged!")
            break
        else:
            iter += 1
            prev_elbo = next_elbo

        
lda()            
end = time.time()
print(end - start)


Iteration: 0 ELBO: -12.418982
12.02796196937561


In [28]:
prof = cProfile.run('lda()', sort=True)

Iteration: 0 ELBO: -12.421164
         16394 function calls (16392 primitive calls) in 12.195 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      319   11.713    0.037   11.713    0.037 {built-in method _pywrap_tensorflow_internal.TFE_Py_Execute}
        1    0.190    0.190    7.068    7.068 <ipython-input-12-23890523edae>:1(elbo)
        1    0.122    0.122   12.176   12.176 <ipython-input-26-0e2d6dc2d4aa>:3(lda)
        1    0.056    0.056    0.582    0.582 <ipython-input-8-371989c98d7e>:2(update_gamma)
        1    0.056    0.056    0.602    0.602 <ipython-input-7-62df51f3fab9>:2(update_lambda)
        1    0.020    0.020    3.800    3.800 <ipython-input-11-b39e0c75cc23>:1(update_phi)
        1    0.019    0.019   12.195   12.195 <string>:1(<module>)
      179    0.001    0.000    0.002    0.000 constant_op.py:89(convert_to_eager_tensor)
     3592    0.001    0.000    0.001    0.000 {built-in method builtins.isinstance

In [454]:
# topic term distribution:
topic_term_dist = np.vstack([(lam / tf.reduce_sum(lam)).numpy()  for lam in lambda_])
topic_term_dist

# doc_topic_dists :array-like, shape (n_docs, n_topics)
doc_topic_dist = tf.stack([tf.reshape(g_k, shape=(g_k.shape[0], )) for g_k in gamma], axis=1)
doc_topic_dist = doc_topic_dist / tf.reduce_sum(doc_topic_dist, axis=1, keep_dims=True)
doc_topic_dist = doc_topic_dist.numpy()

# doc_lengths :array-like, shape n_docs
doc_len = tf.reduce_sum(nw, axis=1)
doc_len = doc_len.numpy()

# vocab :array-like, shape n_terms
vocab = data.iloc[:,0]

# term_frequency :array-like, shape n_terms
term_frec = tf.reduce_sum(nw, axis=0)
term_frec = term_frec.numpy()

In [455]:
import pyLDAvis
topics = pyLDAvis.prepare(topic_term_dist, doc_topic_dist, doc_len, vocab, term_frec)
pyLDAvis.save_html(topics, fileobj="results.html")

/Users/qba/.local/share/virtualenvs/LDA-pHQnYWLy/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


In [451]:
topic_term_dist

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

### Rewrite using updates instead of stacking

In [10]:
ref = tf.fill(value=1.0, dims=(10, ))
ref = tfe.Variable(ref)
upd = tf.fill(value=-1.0, dims=(3, ))
print(ref)
print(tf.scatter_update(ref=ref, indices=tf.convert_to_tensor([0, 1, 2]), updates=upd))
print(ref)

<tf.Variable 'Variable:0' shape=(10,) dtype=float32, numpy=array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)>
tf.Tensor([-1. -1. -1.  1.  1.  1.  1.  1.  1.  1.], shape=(10,), dtype=float32)
<tf.Variable 'Variable:0' shape=(10,) dtype=float32, numpy=array([-1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.], dtype=float32)>


In [11]:
upd

<tf.Tensor: id=102, shape=(3,), dtype=float32, numpy=array([-1., -1., -1.], dtype=float32)>

In [12]:
tf.reduce_sum(ref)

<tf.Tensor: id=117, shape=(), dtype=float32, numpy=4.0>

In [13]:
# initialize LDA parameters
def initialize_variables(K, V, D, alpha=1e-1, eta=1e-1, seed=2014):
    """
    Initialize parameters of LDA model returning adequate Tensors.

    args:
    
        K (int): number of LDA components 
        V (int): vocabulary size
        D (int): number of documents
        alpha (float): hyperparameter for theta prior
        eta (float): hyperparameter for beta prior
       
       
    returns:
    
        eta: [V] tensor with prior parameters (alpha) for beta
        lambda: [V, K] tensor with posterior word distribution per class
        phi: [D, V, K] tensor with vocabulary membership per document
        gamma: [D, K] tensor
        
    """
    tf.set_random_seed(seed)
    
    eta = tfe.Variable(tf.ones(V) * eta, name="eta")
    alpha = tfe.Variable(tf.ones(K) * alpha, name="alpha")
    lambda_ = tfe.Variable(tf.abs(tf.random_normal(shape=(V, K))), name='lambda')
    
    phi = tfe.Variable(tf.random_normal(shape=(D, V, K)), name="phi")
    phi = tf.nn.softmax(phi, axis=2)
    
    gamma = tfe.Variable(tf.abs(tf.random_normal(shape=(D, K))))
    
    return eta, alpha, lambda_, phi, gamma

# test
eta, alpha, lambda_, phi, gamma = initialize_variables(K, V, D)


In [14]:
lambda_

<tf.Variable 'lambda:0' shape=(10, 3) dtype=float32, numpy=
array([[0.30049554, 1.3264463 , 0.3345    ],
       [0.41716772, 0.47891214, 0.60391957],
       [0.52135056, 1.1167537 , 2.0634964 ],
       [0.3132391 , 0.06342854, 0.98612607],
       [0.07255968, 1.0613515 , 0.6355119 ],
       [0.07127264, 1.4961201 , 0.63234806],
       [0.8790525 , 0.26309258, 0.26927882],
       [0.4351647 , 2.0731583 , 0.9537371 ],
       [1.8273319 , 0.38898602, 0.48102027],
       [0.6974121 , 2.812919  , 0.0922653 ]], dtype=float32)>

In [9]:
# lambda update
def update_lambda(lambda_, eta, phi, nw):
    
    K = len(lambda_)
    for k in range(K):
        lambda_[k] = tf.add(tf.reduce_sum(tf.multiply(phi[:,:,k], nw), axis=0, keepdims=True), eta)
        
    return lambda_ 

In [16]:
upd = tf.add(tf.reduce_sum(tf.multiply(phi[:,:,0], nw * 0.0), axis=0, keepdims=False), 0.0)

In [17]:
res = tf.meshgrid(list(range(10)), [0], indexing="ij")
res = tf.squeeze(tf.stack(res, axis=1), axis=2)
res

<tf.Tensor: id=224, shape=(10, 2), dtype=int32, numpy=
array([[0, 0],
       [1, 0],
       [2, 0],
       [3, 0],
       [4, 0],
       [5, 0],
       [6, 0],
       [7, 0],
       [8, 0],
       [9, 0]], dtype=int32)>

In [ ]:
print(lambda_)
lambda_ = tf.scatter_update(
        ref=lambda_, 
        indices=res, 
        updates=tf.add(tf.reduce_sum(tf.multiply(phi[:,:,0], nw * 0.0), axis=0, keepdims=False), 0.0))
print(lambda_)

In [ ]:
tf.scatter_update(ref=lambda_, indices=res, updates=upd)

In [ ]:
?tf.scatter_update